In [10]:
using JuMP, GLPK
using DataFrames

In [21]:
# PARAMETERS
F = 3       # number of fish types
G = 3       # number of fishing grounds
I = 4       # number of processing facilities
J = 110     # number of cities

# DATA: FISH 
fish_price = [201.35, 156.53, 265, 380]


# DATA: FISHING GROUND 
df_fishingground_max = DataFrame(
    Milkfish = [680.94, 3290.55, 3732.93],
    Tilapia = [34.91, 9776.8, 5649.49],
    Shrimp = [0, 2155.94, 5354.92]
)
display(df_fishingground_max)

# DATA: PROCESSING FACILITY
processing_cap = [648000, 127750, 145600, 27000 ]

# DATA: CITIES
budget = [4548.063, 4198.212, 4198.212, 4198.212, 4198.212, 3498.51, 4198.212, 4198.212, 4198.212, 4198.212, 4198.212, 4198.212, 3498.51, 4198.212, 4198.212, 3498.51, 3498.51, 3498.51, 4198.212, 3498.51, 4198.212, 4198.212, 4198.212, 3498.51, 4198.212, 3498.51, 4198.212, 4198.212, 3498.51, 3498.51, 4198.212, 4198.212, 3498.51, 3498.51, 3498.51, 3498.51, 4198.212, 4198.212, 3498.51, 3498.51, 4198.212, 4198.212, 4198.212, 4198.212, 3498.51, 4198.212, 4198.212, 4198.212, 3498.51, 3498.51, 4198.212, 3498.51, 3498.51, 4198.212, 4198.212, 3498.51, 3498.51, 3498.51, 3498.51, 4198.212, 3498.51, 3498.51, 4198.212, 4198.212, 3498.51, 4198.212, 4198.212, 4198.212, 3498.51, 3498.51, 3498.51, 4198.212, 4198.212, 4198.212, 3498.51, 3498.51, 4198.212, 3498.51, 3498.51, 3498.51, 3498.51, 3498.51, 3498.51, 4198.212, 4198.212, 3498.51, 4198.212, 4198.212, 4198.212, 3498.51, 3498.51, 3498.51, 3498.51, 3498.51, 4198.212, 3498.51, 3498.51, 4198.212, 3498.51, 3498.51]
population = [24922000, 2960048, 2100000, 1776949, 1661584, 1273219, 964169, 887399, 886722, 803159, 728402, 714978, 703141, 697315, 689992, 664625, 654220, 651813, 629616, 606293, 600783, 574000, 543445, 539671, 497604, 496794, 462928, 456059, 450583, 443954, 440656, 425758, 414812, 407437, 388305, 386451, 385398, 380522, 376933, 372931, 372910, 366358, 364116, 363115, 355330, 354666, 351437, 342769, 327325, 326001, 325079, 313631, 312116, 307079, 296202, 295644, 293244, 289820, 285348, 278924, 273306, 263048, 261189, 260317, 254453, 251881, 247543, 230998, 225673, 218500, 216546, 210503, 210452, 209533, 209230, 207314, 207010, 205424, 203598, 200198, 195398, 193936, 191210, 190712, 188376, 186960, 182237, 179292, 174302, 173502, 173403, 172605, 172433, 172073, 171107, 169953, 168470, 166334, 161868, 160987]

# DATA FOR OBJECTIVE FUNCTION



Row,Milkfish,Tilapia,Shrimp
,Float64,Float64,Float64
1,680.94,34.91,0.0
2,3290.55,9776.8,2155.94
3,3732.93,5649.49,5354.92


4-element Vector{Int64}:
 648000
 127750
 145600
  27000

In [9]:
# VARIABLES
m = Model(GLPK.Optimizer)
@variable(m, x[1:F, 1:G, 1:I])
@variable(m, y[1:F, 1:I, 1:J])
@variable(m, z[1:I], Bin)
@variable(m, s[1:J])

110-element Vector{VariableRef}:
 s[1]
 s[2]
 s[3]
 s[4]
 s[5]
 s[6]
 s[7]
 s[8]
 s[9]
 s[10]
 s[11]
 s[12]
 s[13]
 ⋮
 s[99]
 s[100]
 s[101]
 s[102]
 s[103]
 s[104]
 s[105]
 s[106]
 s[107]
 s[108]
 s[109]
 s[110]

In [22]:
# CONSTRAINTS
# Fishing Restriction Constraint
for f in 1:F
    for g in 1:G
        @constraint(m, sum(x[f, g, i] for i in 1:I) <= df_fishingground_max[g, f])
    end
end

# Distribution/Processing Capacity Constraint
for i in 1:I
    @constraint(m, sum(y[f, i, j] for f in 1:F for j in 1:J) <= processing_cap[i])
end

# Min/Max Fish Consumption in each City 
for j in 1:J # arbitrary number for now
    @constraint(m, sum(y[f, i, j] for f in 1:F for i in 1:I) <= 100000000) 
    @constraint(m, sum(y[f, i, j] for f in 1:F for i in 1:I) >= 0)
end
 
# Total Fish Distributed < Total Fish Caught
@constraint(m, sum(y[f, i, j] for f in 1:F for i in 1:I for j in 1:J) <= sum(x[f, g, i] for f in 1:F for g in 1:G for i in 1:I))

# Budget Constraint
for j in 1:J
    @constraint(m, sum(y[f, i, j] for f in 1:F for i in 1:I) * fish_price[f] <= population[j] * budget[j] + s[j])
end

# Indicator variables
# If we close facility i, we cannot send any fish to facility i
for f in 1:F
    for g in 1:G
        for i in 1:I
            @constraint(m, x[f,g,i] <= z[i] * df_fishingground_max[g, f])
        end
    end
end

# Indicator variables
# If we close facility i, we cannot send any fish from facility i
for f in 1:F
    for i in 1:I
        for j in 1:J
            @constraint(m, y[f,i,j] <= z[i] * processing_cap[i])
        end
    end
end

In [ ]:
# OBJECTIVE FUNCTION
@objective(m, Max, sum(value[f]*y[f, i, j] for f in 1:F for i in 1:I for j in 1:J) - sum(fishing_cost[g, i]*x[f, g, i] for f in 1:F for g in 1:G for i in 1:I) - sum(transport_cost[i,j]*y[f, i, j] for f in 1:F for i in 1:I for j in 1:J) - sum(operational_cost[i]*z[i] for i in 1:I) - sum(s[j] for j in 1:J))